## Pré Processamento - Parte 2

#### Nesse notebook serão realizadas mais etapas de pré processamento: 

- Análise de percentual de nulos em cada coluna 
- Escolha das colunas a serem mantidas na análise
- Renomear as colunas do questionário para nomes mais intuitivos
- Criar um dataframe numérico, aplicando o eequivalente a um Ordinal Enconder
- Transformar as colunas categóricas que estão como numéricas, em colunas de fato categóricas
- Criar um dataframe categórico com os mapeamentos 


### Análise de percentual de nulos em cada coluna

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
df = pd.read_parquet("df_enem_otimizado.parquet")
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,210061959676,2023,12,M,1,1,1,1,6,1,...,B,A,A,C,A,B,E,A,A,B
3933951,210061950911,2023,1,F,1,1,2,3,0,1,...,B,A,B,C,B,B,B,B,C,B
3933952,210061965966,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,B
3933953,210061932304,2023,2,M,1,1,1,2,0,3,...,B,B,B,C,A,A,D,A,C,B


Análise se há algum número de inscrição duplicado, que deve ser retirado

In [17]:
df['NU_INSCRICAO'].value_counts().sort_values(ascending=False)

210059085136    1
210060460428    1
210058061537    1
210059388087    1
210061103945    1
               ..
210061959674    1
210061959675    1
210061959676    1
210061950911    1
210058924455    1
Name: NU_INSCRICAO, Length: 3933955, dtype: int64

In [18]:
df['NU_INSCRICAO'].nunique() == len(df)

True

In [19]:
# Contagem de nulos
pd.DataFrame(df.isnull().sum()).sort_values(ascending=False, by=0).head(32)

,0
TP_LOCALIZACAO_ESC,2975449
CO_UF_ESC,2975449
TP_SIT_FUNC_ESC,2975449
TP_DEPENDENCIA_ADM_ESC,2975449
SG_UF_ESC,2975449
CO_MUNICIPIO_ESC,2975449
NO_MUNICIPIO_ESC,2975449
TP_ENSINO,2594874
NU_NOTA_CN,1241528
CO_PROVA_MT,1241528


Muitos dos valores acima, principalmente em relação as notas são de candidatos que faltaram

###

### Escolha das colunas a serem mantidas na análise

#### Agora vamos analisar todas as colunas do dataframe com o auxílio tabela "Dicionário_Microdados_Enem_2023.ods" que se encontra na pasta "DICIONÁRIO" para decidir quais colunas são de fato úteis para a nossa análise e para responder as perguntas que queremos investigar na etapa de análise exploratória (EDA)

In [20]:
df.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

#### Ao analisar o dicionário de microdados, foi decidido que as seguintes colunas serão mantidas

In [21]:
manter_colunas = ['TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',
       'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013',
       'Q014', 'Q015', 'Q016', 'Q017', 'Q019', 'Q022', 'Q024', 'Q025']

Portanto, vamos remover as colunas que não estão na lista acima

In [22]:
drop_columns =[]
for coluna in df.columns:
    if coluna not in manter_colunas:
        drop_columns.append(coluna)

drop_columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_NACIONALIDADE',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'Q018',
 'Q020',
 'Q021',
 'Q023']

In [23]:
df.drop(columns = drop_columns, inplace = True)
df

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,Q012,Q013,Q014,Q015,Q016,Q017,Q019,Q022,Q024,Q025
0,14,M,2,1,1,17,1,NaN,0,NaN,...,C,B,B,D,C,C,B,B,A,B
1,12,M,2,1,1,16,1,NaN,0,NaN,...,B,B,B,A,B,A,B,C,D,B
2,6,F,1,1,1,0,1,NaN,0,NaN,...,B,A,B,A,B,A,B,A,A,B
3,2,F,1,3,2,0,2,1.0,0,CE,...,B,A,A,A,A,A,B,D,A,B
4,3,F,1,3,2,0,2,1.0,0,CE,...,B,A,A,A,A,A,B,B,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,12,M,1,1,1,6,1,NaN,0,NaN,...,B,A,B,B,B,A,C,E,A,B
3933951,1,F,1,1,3,0,1,NaN,1,NaN,...,B,B,B,A,B,A,C,B,C,B
3933952,3,F,1,3,2,0,2,1.0,0,NaN,...,B,A,A,A,A,A,B,B,A,B
3933953,2,M,1,1,2,0,3,1.0,0,NaN,...,C,B,B,A,B,B,C,D,C,B


In [24]:
df.columns

Index(['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015',
       'Q016', 'Q017', 'Q019', 'Q022', 'Q024', 'Q025'],
      dtype='object')

### Renomar as colunas:

#### Da análise do questionário socioeconômico, vamos renomear as colunas para que façam mais sentido

In [25]:
rename_columns = {
    'Q001': 'escolaridade_pai',
    'Q002': 'escolaridade_mae',
    'Q003': 'ocupacao_pai',
    'Q004': 'ocupacao_mae',
    'Q005': 'qtd_pessoas_residencia',
    'Q006': 'renda_familiar',
    'Q007': 'empregado_domestico',
    'Q008': 'banheiro',
    'Q009': 'qtd_quartos',
    'Q010': 'carro',
    'Q011': 'motocicleta',
    'Q012': 'geladeira',
    'Q013': 'freezer',
    'Q014': 'maquina_lavar_roupa',
    'Q015': 'maquina_secar_roupa',
    'Q016': 'microondas',
    'Q017': 'maquina_lavar_louca',
    'Q019': 'televisao',
    'Q022': 'celular',
    'Q024': 'computador',
    'Q025': 'internet'
}

# Renomeando as colunas no DataFrame
df.rename(columns=rename_columns, inplace=True)

# Exibindo o DataFrame atualizado para confirmar
df

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,geladeira,freezer,maquina_lavar_roupa,maquina_secar_roupa,microondas,maquina_lavar_louca,televisao,celular,computador,internet
0,14,M,2,1,1,17,1,NaN,0,NaN,...,C,B,B,D,C,C,B,B,A,B
1,12,M,2,1,1,16,1,NaN,0,NaN,...,B,B,B,A,B,A,B,C,D,B
2,6,F,1,1,1,0,1,NaN,0,NaN,...,B,A,B,A,B,A,B,A,A,B
3,2,F,1,3,2,0,2,1.0,0,CE,...,B,A,A,A,A,A,B,D,A,B
4,3,F,1,3,2,0,2,1.0,0,CE,...,B,A,A,A,A,A,B,B,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,12,M,1,1,1,6,1,NaN,0,NaN,...,B,A,B,B,B,A,C,E,A,B
3933951,1,F,1,1,3,0,1,NaN,1,NaN,...,B,B,B,A,B,A,C,B,C,B
3933952,3,F,1,3,2,0,2,1.0,0,NaN,...,B,A,A,A,A,A,B,B,A,B
3933953,2,M,1,1,2,0,3,1.0,0,NaN,...,C,B,B,A,B,B,C,D,C,B


### Ordinal Encoding - Criação de Dataframe Numérico

#### Vamos fazer o equivalente a um ordinal encoding nas colunas abaixo para que possamos criar um dataframe numérico, que será muito importante na etapa de Machine Learning, que não reconhece variáveis categóricas e precisa de processamentos como OneHotEnconder para lidar com elas

#### Nessa etapa, podemos fazer essa transformação, pois os números das colunas que vamos transformar, seguem uma ideia de 'ranking', então na prática, o que temos é um Ordinal Encoding

In [26]:
colunas_mapeadas = ['escolaridade_pai', 'escolaridade_mae', 'renda_familiar', 'empregado_domestico', 'banheiro', 'qtd_quartos',
       'carro', 'motocicleta', 'geladeira', 'freezer', 'maquina_lavar_roupa',
       'maquina_secar_roupa', 'microondas', 'maquina_lavar_louca', 'televisao',
       'celular', 'computador', 'internet']
#map_itens = {'A': 0, 'B': 1, 'B': 2, 'C': 3, 'D': 4}
map_itens = {"A": 0,"B": 1,"C": 2,"D": 3,"E": 4,"F": 5,"G": 6,"H": 7,"I": 8,"J": 9,"K": 10,"L": 11,"M": 12,"N": 13,"O": 14,"P": 15,"Q": 16}


for coluna in colunas_mapeadas:
    df[coluna] = df[coluna].map(map_itens)

df

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,geladeira,freezer,maquina_lavar_roupa,maquina_secar_roupa,microondas,maquina_lavar_louca,televisao,celular,computador,internet
0,14,M,2,1,1,17,1,NaN,0,NaN,...,2,1,1,3,2,2,1,1,0,1
1,12,M,2,1,1,16,1,NaN,0,NaN,...,1,1,1,0,1,0,1,2,3,1
2,6,F,1,1,1,0,1,NaN,0,NaN,...,1,0,1,0,1,0,1,0,0,1
3,2,F,1,3,2,0,2,1.0,0,CE,...,1,0,0,0,0,0,1,3,0,1
4,3,F,1,3,2,0,2,1.0,0,CE,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,12,M,1,1,1,6,1,NaN,0,NaN,...,1,0,1,1,1,0,2,4,0,1
3933951,1,F,1,1,3,0,1,NaN,1,NaN,...,1,1,1,0,1,0,2,1,2,1
3933952,3,F,1,3,2,0,2,1.0,0,NaN,...,1,0,0,0,0,0,1,1,0,1
3933953,2,M,1,1,2,0,3,1.0,0,NaN,...,2,1,1,0,1,1,2,3,2,1


#### Vamos analisar os tipos para verificar se as colunas que utilizamos estão como numéricas, caso não esteja, vamos transformar para um tipo numérico otimizado

In [27]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 41 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   TP_FAIXA_ETARIA         int8    
 1   TP_SEXO                 category
 2   TP_ESTADO_CIVIL         int8    
 3   TP_COR_RACA             int8    
 4   TP_ST_CONCLUSAO         int8    
 5   TP_ANO_CONCLUIU         int8    
 6   TP_ESCOLA               int8    
 7   TP_ENSINO               float16 
 8   IN_TREINEIRO            int8    
 9   SG_UF_ESC               category
 10  TP_DEPENDENCIA_ADM_ESC  float16 
 11  TP_LOCALIZACAO_ESC      float16 
 12  SG_UF_PROVA             category
 13  TP_PRESENCA_CN          int8    
 14  TP_PRESENCA_CH          int8    
 15  NU_NOTA_CN              float64 
 16  NU_NOTA_CH              float64 
 17  NU_NOTA_LC              float64 
 18  NU_NOTA_MT              float64 
 19  NU_NOTA_REDACAO         float16 
 20  escolaridade_pai        category
 21  escolari

In [28]:
for coluna in colunas_mapeadas:
    df[coluna] = df[coluna].astype('float16')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 41 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   TP_FAIXA_ETARIA         int8    
 1   TP_SEXO                 category
 2   TP_ESTADO_CIVIL         int8    
 3   TP_COR_RACA             int8    
 4   TP_ST_CONCLUSAO         int8    
 5   TP_ANO_CONCLUIU         int8    
 6   TP_ESCOLA               int8    
 7   TP_ENSINO               float16 
 8   IN_TREINEIRO            int8    
 9   SG_UF_ESC               category
 10  TP_DEPENDENCIA_ADM_ESC  float16 
 11  TP_LOCALIZACAO_ESC      float16 
 12  SG_UF_PROVA             category
 13  TP_PRESENCA_CN          int8    
 14  TP_PRESENCA_CH          int8    
 15  NU_NOTA_CN              float64 
 16  NU_NOTA_CH              float64 
 17  NU_NOTA_LC              float64 
 18  NU_NOTA_MT              float64 
 19  NU_NOTA_REDACAO         float16 
 20  escolaridade_pai        float16 
 21  escolari

#### Vamos salvar esse dataframe numérico em parquet

In [29]:
df.to_parquet("df_num.parquet", compression= "gzip")

Testando se funcionou:

In [30]:
pd.read_parquet("df_num.parquet").info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 41 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   TP_FAIXA_ETARIA         int8    
 1   TP_SEXO                 category
 2   TP_ESTADO_CIVIL         int8    
 3   TP_COR_RACA             int8    
 4   TP_ST_CONCLUSAO         int8    
 5   TP_ANO_CONCLUIU         int8    
 6   TP_ESCOLA               int8    
 7   TP_ENSINO               float16 
 8   IN_TREINEIRO            int8    
 9   SG_UF_ESC               category
 10  TP_DEPENDENCIA_ADM_ESC  float16 
 11  TP_LOCALIZACAO_ESC      float16 
 12  SG_UF_PROVA             category
 13  TP_PRESENCA_CN          int8    
 14  TP_PRESENCA_CH          int8    
 15  NU_NOTA_CN              float64 
 16  NU_NOTA_CH              float64 
 17  NU_NOTA_LC              float64 
 18  NU_NOTA_MT              float64 
 19  NU_NOTA_REDACAO         float16 
 20  escolaridade_pai        float16 
 21  escolari

### Criação de Dataframe Categórico

#### Como temos diversas colunas categóricas que estão sendo utilizadas como numéricas vamos criar um cópia do dataframe para fazer o mapeamento desses categóricos de acordo com os dados contidos do dicionário

#### Primeiro será o mapeamento das informações gerais, e em seguida, o mapeamento dos dados socioeconômicos. 

#### Além disso, vamos mudar o nome das colunas desse questionário socioeconômico para nomes que façam mais sentido e nos ajude a visualizar as informações de forma mais simples durante as análises

In [31]:
def remover_colunas(df):
    df = pd.read_parquet("df_enem_otimizado.parquet")
    df['NU_INSCRICAO'].value_counts().sort_values(ascending=False)
    df['NU_INSCRICAO'].nunique() == len(df)

    #### Ao analisar o dicionário de microdados, foi decidido que as seguintes colunas serão mantidas
    manter_colunas = ['TP_FAIXA_ETARIA', 'TP_SEXO',
        'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ST_CONCLUSAO',
        'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
        'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
        'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
        'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',
        'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013',
        'Q014', 'Q015', 'Q016', 'Q017', 'Q019', 'Q022', 'Q024', 'Q025']
    
    #Revomendo as colunas da lista acima
    drop_columns =[]
    for coluna in df.columns:
        if coluna not in manter_colunas:
            drop_columns.append(coluna)

    drop_columns
    df.drop(columns = drop_columns, inplace = True)
    return df

In [32]:
df = pd.read_parquet("df_enem_otimizado.parquet")
df = remover_colunas (df)
df_mapped = df.copy()

In [33]:
# Mapeamento de dados generalistas

df_mapped['TP_FAIXA_ETARIA'] = df_mapped['TP_FAIXA_ETARIA'].map({
1:	'< 17 ',
2:	'17 ',
3:	'18 ',
4:	'19 ',
5:	'20 ',
6:	'21 ',
7:	'22 ',
8:	'23 ',
9:	'24 ',
10:	'25 ',
11:	'>=26 <= 30',
12:	'>=31 <= 35',
13:	'>=36 <= 40',
14:	'>=41 <= 45',
15:	'>=46 <= 50',
16:	'>=51 <= 55',
17:	'>=56 <= 60',
18:	'>=61 <= 65',
19:	'>=66 <= 70',
20:	'> 70 '
})


df_mapped['TP_ESTADO_CIVIL'] = df_mapped['TP_ESTADO_CIVIL'].map({
0 : 'Não informado',
1 : 'Solteiro(a)',
2 : 'Casado(a)/Mora com companheiro(a)',
3 : 'Divorciado(a)/Desquitado(a)/Separado(a)',
4 : 'Viúvo(a)'
})


df_mapped['TP_COR_RACA'] = df_mapped['TP_COR_RACA'].map({
0 : 'Não declarado',
1 : 'Branco(a)',
2 : 'Preto(a)',
3 : 'Pardo(a)',
4 : 'Amarelo(a)',
5 : 'Indigena'
})


df_mapped['TP_ST_CONCLUSAO'] =	df_mapped['TP_ST_CONCLUSAO'].map({
1 : 'Ensino Médio Concluído',
2 : 'Cursando Ensino Médio, conclusão em 2023',
3 : 'Cursando Ensino Médio, conclusão após 2023',
4 : 'Ensino Médio não concluído e não sendo cursado'
})

# TP_ANO_CONCLUIU
df_mapped['TP_ANO_CONCLUIU'] = df_mapped['TP_ANO_CONCLUIU'].map({
    0: 'Não informado',
    1: '2022',
    2: '2021',
    3: '2020',
    4: '2019',
    5: '2018',
    6: '2017',
    7: '2016',
    8: '2015',
    9: '2014',
    10: '2013',
    11: '2012',
    12: '2011',
    13: '2010',
    14: '2009',
    15: '2008',
    16: '2007',
    17: 'Antes de 2007'
})

# TP_ESCOLA
df_mapped['TP_ESCOLA'] = df_mapped['TP_ESCOLA'].map({
    1: 'Não Respondeu',
    2: 'Pública',
    3: 'Privada'
})

# TP_ENSINO
df_mapped['TP_ENSINO'] = df_mapped['TP_ENSINO'].map({
    1: 'Ensino Regular',
    2: 'Educação Especial - Modalidade Substitutiva'
})

# IN_TREINEIRO
df_mapped['IN_TREINEIRO'] = df_mapped['IN_TREINEIRO'].map({
    1: 'Sim',
    0: 'Não'
})

# TP_DEPENDENCIA_ADM_ESC
df_mapped['TP_DEPENDENCIA_ADM_ESC'] = df_mapped['TP_DEPENDENCIA_ADM_ESC'].map({
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal',
    4: 'Privada'
})

# TP_LOCALIZACAO_ESC
df_mapped['TP_LOCALIZACAO_ESC'] = df_mapped['TP_LOCALIZACAO_ESC'].map({
    1: 'Urbana',
    2: 'Rural'
})

# TP_PRESENCA_CN, TP_PRESENCA_CH, TP_PRESENCA_LC, TP_PRESENCA_MT
df_mapped['TP_PRESENCA_CN'] = df_mapped['TP_PRESENCA_CN'].map({
    0: 'Faltou à prova',
    1: 'Presente na prova',
    2: 'Eliminado na prova'
})

df_mapped['TP_PRESENCA_CH'] = df_mapped['TP_PRESENCA_CH'].map({
    0: 'Faltou à prova',
    1: 'Presente na prova',
    2: 'Eliminado na prova'
})


df_mapped

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,Q012,Q013,Q014,Q015,Q016,Q017,Q019,Q022,Q024,Q025
0,>=41 <= 45,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,Antes de 2007,Não Respondeu,NaN,Não,NaN,...,C,B,B,D,C,C,B,B,A,B
1,>=31 <= 35,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,2007,Não Respondeu,NaN,Não,NaN,...,B,B,B,A,B,A,B,C,D,B
2,21,F,Solteiro(a),Branco(a),Ensino Médio Concluído,Não informado,Não Respondeu,NaN,Não,NaN,...,B,A,B,A,B,A,B,A,A,B
3,17,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,B,A,A,A,A,A,B,D,A,B
4,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,B,A,A,A,A,A,B,B,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,>=31 <= 35,M,Solteiro(a),Branco(a),Ensino Médio Concluído,2017,Não Respondeu,NaN,Não,NaN,...,B,A,B,B,B,A,C,E,A,B
3933951,< 17,F,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão após 2023",Não informado,Não Respondeu,NaN,Sim,NaN,...,B,B,B,A,B,A,C,B,C,B
3933952,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,NaN,...,B,A,A,A,A,A,B,B,A,B
3933953,17,M,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Privada,Ensino Regular,Não,NaN,...,C,B,B,A,B,B,C,D,C,B


In [34]:
# Mapeamento SocioEconômico 

df_mapped['Q001'] = df_mapped['Q001'].map({
    'A': 'Nunca estudou',
    'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental',
    'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental',
    'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio',
    'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
    'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
    'G': 'Completou a Pós-graduação',
    'H': 'Não sei'
})

df_mapped['Q002'] = df_mapped['Q002'].map({
    'A': 'Nunca estudou',
    'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental',
    'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental',
    'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio',
    'E': 'Completou o Ensino Médio, mas não completou a Faculdade',
    'F': 'Completou a Faculdade, mas não completou a Pós-graduação',
    'G': 'Completou a Pós-graduação',
    'H': 'Não sei'
})

df_mapped['Q003'] = df_mapped['Q003'].map({
    'A': 'Grupo 1: Lavrador, agricultor sem empregados, bóia fria, criador de animais, pescador, etc.',
    'B': 'Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, motorista particular, etc.',
    'C': 'Grupo 3: Padeiro, cozinheiro industrial, operário de fábrica, pedreiro, eletricista, etc.',
    'D': 'Grupo 4: Professor (ensino fundamental/médio), técnico, policial, gerente, microempresário, etc.',
    'E': 'Grupo 5: Médico, engenheiro, advogado, juiz, professor universitário, político, etc.',
    'F': 'Não sei'
})

df_mapped['Q004'] = df_mapped['Q004'].map({
    'A': 'Grupo 1: Lavradora, agricultora sem empregados, bóia fria, criadora de animais, pescadora, etc.',
    'B': 'Grupo 2: Diarista, empregada doméstica, cuidadora de idosos, babá, motorista particular, etc.',
    'C': 'Grupo 3: Padeira, cozinheira industrial, operária de fábrica, pedreira, eletricista, etc.',
    'D': 'Grupo 4: Professora (ensino fundamental/médio), técnica, policial, gerente, microempresária, etc.',
    'E': 'Grupo 5: Médica, engenheira, advogada, juíza, professora universitária, política, etc.',
    'F': 'Não sei'
})

#df_mapped['Q005'] = df_mapped['Q005'].map({str(i): str(i) for i in range(1, 21)})

df_mapped['Q006'] = df_mapped['Q006'].map({
    'A': 'Nenhuma Renda',
    'B': 'Até R$ 1.320,00',
    'C': 'De R$ 1.320,01 até R$ 1.980,00',
    'D': 'De R$ 1.980,01 até R$ 2.640,00',
    'E': 'De R$ 2.640,01 até R$ 3.300,00',
    'F': 'De R$ 3.300,01 até R$ 3.960,00',
    'G': 'De R$ 3.960,01 até R$ 5.280,00',
    'H': 'De R$ 5.280,01 até R$ 6.600,00',
    'I': 'De R$ 6.600,01 até R$ 7.920,00',
    'J': 'De R$ 7.920,01 até R$ 9.240,00',
    'K': 'De R$ 9.240,01 até R$ 10.560,00',
    'L': 'De R$ 10.560,01 até R$ 11.880,00',
    'M': 'De R$ 11.880,01 até R$ 13.200,00',
    'N': 'De R$ 13.200,01 até R$ 15.840,00',
    'O': 'De R$ 15.840,01 até R$ 19.800,00',
    'P': 'De R$ 19.800,01 até R$ 26.400,00',
    'Q': 'Acima de R$ 26.400,00'
})

df_mapped['Q007'] = df_mapped['Q007'].map({
    'A': 'Não',
    'B': 'Sim, um ou dois dias por semana',
    'C': 'Sim, três ou quatro dias por semana',
    'D': 'Sim, pelo menos cinco dias por semana'
})

df_mapped['Q008'] = df_mapped['Q008'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q009'] = df_mapped['Q009'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q010'] = df_mapped['Q010'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q011'] = df_mapped['Q011'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q012'] = df_mapped['Q012'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q013'] = df_mapped['Q013'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q014'] = df_mapped['Q014'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q015'] = df_mapped['Q015'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q016'] = df_mapped['Q016'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q017'] = df_mapped['Q017'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q019'] = df_mapped['Q019'].map({
    'A': 'Não',
    'B': 'Sim, uma',
    'C': 'Sim, duas',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q022'] = df_mapped['Q022'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q024'] = df_mapped['Q024'].map({
    'A': 'Não',
    'B': 'Sim, um',
    'C': 'Sim, dois',
    'D': 'Sim, três',
    'E': 'Sim, quatro ou mais'
})

df_mapped['Q025'] = df_mapped['Q025'].map({
    'A': 'Não',
    'B': 'Sim'
})

rename_columns = {
    'Q001': 'escolaridade_pai',
    'Q002': 'escolaridade_mae',
    'Q003': 'ocupacao_pai',
    'Q004': 'ocupacao_mae',
    'Q005': 'qtd_pessoas_residencia',
    'Q006': 'renda_familiar',
    'Q007': 'empregado_domestico',
    'Q008': 'banheiro',
    'Q009': 'qtd_quartos',
    'Q010': 'carro',
    'Q011': 'motocicleta',
    'Q012': 'geladeira',
    'Q013': 'freezer',
    'Q014': 'maquina_lavar_roupa',
    'Q015': 'maquina_secar_roupa',
    'Q016': 'microondas',
    'Q017': 'maquina_lavar_louca',
    'Q019': 'televisao',
    'Q022': 'celular',
    'Q024': 'computador',
    'Q025': 'internet'
}

# Renomeando as colunas no DataFrame
df_mapped.rename(columns=rename_columns, inplace=True)

# Exibindo o DataFrame atualizado para confirmar
df_mapped

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,geladeira,freezer,maquina_lavar_roupa,maquina_secar_roupa,microondas,maquina_lavar_louca,televisao,celular,computador,internet
0,>=41 <= 45,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,Antes de 2007,Não Respondeu,NaN,Não,NaN,...,"Sim, duas","Sim, um","Sim, uma","Sim, três","Sim, dois","Sim, duas","Sim, uma","Sim, um",Não,Sim
1,>=31 <= 35,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,2007,Não Respondeu,NaN,Não,NaN,...,"Sim, uma","Sim, um","Sim, uma",Não,"Sim, um",Não,"Sim, uma","Sim, dois","Sim, três",Sim
2,21,F,Solteiro(a),Branco(a),Ensino Médio Concluído,Não informado,Não Respondeu,NaN,Não,NaN,...,"Sim, uma",Não,"Sim, uma",Não,"Sim, um",Não,"Sim, uma",Não,Não,Sim
3,17,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, três",Não,Sim
4,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, um",Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,>=31 <= 35,M,Solteiro(a),Branco(a),Ensino Médio Concluído,2017,Não Respondeu,NaN,Não,NaN,...,"Sim, uma",Não,"Sim, uma","Sim, uma","Sim, um",Não,"Sim, duas","Sim, quatro ou mais",Não,Sim
3933951,< 17,F,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão após 2023",Não informado,Não Respondeu,NaN,Sim,NaN,...,"Sim, uma","Sim, um","Sim, uma",Não,"Sim, um",Não,"Sim, duas","Sim, um","Sim, dois",Sim
3933952,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,NaN,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, um",Não,Sim
3933953,17,M,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Privada,Ensino Regular,Não,NaN,...,"Sim, duas","Sim, um","Sim, uma",Não,"Sim, um","Sim, uma","Sim, duas","Sim, três","Sim, dois",Sim


In [35]:
df_mapped.to_parquet("df_mapped.parquet", compression= "gzip")

Testando se Funcionou

In [36]:
pd.read_parquet("df_mapped.parquet")

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,...,geladeira,freezer,maquina_lavar_roupa,maquina_secar_roupa,microondas,maquina_lavar_louca,televisao,celular,computador,internet
0,>=41 <= 45,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,Antes de 2007,Não Respondeu,None,Não,NaN,...,"Sim, duas","Sim, um","Sim, uma","Sim, três","Sim, dois","Sim, duas","Sim, uma","Sim, um",Não,Sim
1,>=31 <= 35,M,Casado(a)/Mora com companheiro(a),Branco(a),Ensino Médio Concluído,2007,Não Respondeu,None,Não,NaN,...,"Sim, uma","Sim, um","Sim, uma",Não,"Sim, um",Não,"Sim, uma","Sim, dois","Sim, três",Sim
2,21,F,Solteiro(a),Branco(a),Ensino Médio Concluído,Não informado,Não Respondeu,None,Não,NaN,...,"Sim, uma",Não,"Sim, uma",Não,"Sim, um",Não,"Sim, uma",Não,Não,Sim
3,17,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, três",Não,Sim
4,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,CE,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, um",Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,>=31 <= 35,M,Solteiro(a),Branco(a),Ensino Médio Concluído,2017,Não Respondeu,None,Não,NaN,...,"Sim, uma",Não,"Sim, uma","Sim, uma","Sim, um",Não,"Sim, duas","Sim, quatro ou mais",Não,Sim
3933951,< 17,F,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão após 2023",Não informado,Não Respondeu,None,Sim,NaN,...,"Sim, uma","Sim, um","Sim, uma",Não,"Sim, um",Não,"Sim, duas","Sim, um","Sim, dois",Sim
3933952,18,F,Solteiro(a),Pardo(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Pública,Ensino Regular,Não,NaN,...,"Sim, uma",Não,Não,Não,Não,Não,"Sim, uma","Sim, um",Não,Sim
3933953,17,M,Solteiro(a),Branco(a),"Cursando Ensino Médio, conclusão em 2023",Não informado,Privada,Ensino Regular,Não,NaN,...,"Sim, duas","Sim, um","Sim, uma",Não,"Sim, um","Sim, uma","Sim, duas","Sim, três","Sim, dois",Sim
